# Data Analysis 1

In [1]:
#importing libraries
import os
import nltk
import re
import numpy as np
import regex
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import pandas as pd
import string
import matplotlib.pyplot as plt
import seaborn as sns

# Functions

In [2]:
#This function removes all of the rows that contain a string in the passed column
def rmStr (df, col):
    row_index=0
    row_ind=[]
    for i in df[col]:
        try:
            int(i)
        except:
            row_ind.append(row_index)
        row_index+=1
    return df.drop(labels=row_ind,axis=0)

In [3]:
# function that finds the unique values in a list
def get_uniques(x):
    return list(x.unique())

# Importing csv files

In [4]:
loose=pd.read_csv('original-sheets\Responses-Loose.csv')
student_info=pd.read_csv('corpus-files\student_information.csv')
course=pd.read_csv('corpus-files\course.csv')
answer_df=pd.read_csv('answer.csv',index_col = 'answer_id', low_memory=False)


In [5]:
#dropping to only relevant columns
loose=loose.drop(columns=['3Langs','Item','gender','course_id','class_id','version','text1_len','text2_len','text1',
                         'text2 (line breaks/extra spaces removed, spaces added to reach 60)','Judgement','Notes'])

In [6]:
course_=course.drop(columns=['class_id','semester','section'])

student=student_info.drop(columns=['birth_year','language_used_at_home','non_native_language_1','yrs_of_study_lang1',
                                  'study_in_classroom_lang1','ways_of_study_lang1','non_native_language_2','study_in_classroom_lang2',
                                  'ways_of_study_lang2','non_native_language_3','yrs_of_study_lang3','study_in_classroom_lang3',
                                  'ways_of_study_lang3','yrs_of_english_learning','yrs_in_english_environment','yrs_of_study_lang2'])
answer_=answer_df[['question_id','anon_id','course_id','version','text_len','text','tokens']].reset_index()

In [7]:
#Cleaning the answer df
col='course_id'
answer=rmStr(answer_, col) #removes strings from answer_.course_id 

In [38]:
# Merging the datasets

merge_ans=answer[['anon_id','course_id']] #sets answer df up for merging 
student_merge=student.drop(columns=['course_history']) #set student df up for merging

student_ans=student_merge.merge(answer,on='anon_id').astype({'course_id':'int64'}) #merges student and answers

stu_ans_crs_=student_ans.merge(course_, on='course_id').drop(columns=['age','course_id','version','tokens']) #merges the student-answers df with course df

# Cleaning Ans in Whole Dataset

In [39]:
# adding spaces after punctuation if needed

index=0
stu_ans_crs=stu_ans_crs_.copy()
stu_ans_crs['processed_text']=pd.NaT
stu_ans_crs['wordtype_len']=pd.NaT
words=[]
for text in stu_ans_crs.text:
    uwords=[]
    text=text.replace('\n',' ')
    nopunc=text.translate(str.maketrans('','',string.punctuation))
    nopunc=nopunc.lower()
    stu_ans_crs.at[index,'processed_text']=nopunc
    words=nopunc.strip().split(" ")
    for word in words:
        if word=='':
            words.remove(word)
        elif word not in uwords:
            uwords.append(word)
    stu_ans_crs.at[index,'text_len']=len(words)
    stu_ans_crs.at[index,'wordtype_len']=len(uwords)
    index+=1

In [40]:
#dropping all rows where the answer is less than 60 words
index=0
ind_list=[]
for i in stu_ans_crs.text_len:
    txtlen=int(i)
    if txtlen < 60:
        ind_list.append(index)
    index+=1

stu_ans_crs=stu_ans_crs.drop(labels=ind_list,axis=0).reset_index()


In [41]:
stu_ans_crs

,index,anon_id,gender,native_language,answer_id,question_id,text_len,text,level_id,processed_text,wordtype_len
0,0,do6,Female,Russian,150,4,299,Some people prefer eat out and some like doing...,5,some people prefer eat out and some like doing...,125
1,1,do6,Female,Russian,1221,97,288,My opinion is that a person does need educatio...,5,my opinion is that a person does need educatio...,112
2,2,do6,Female,Russian,1957,189,317,There are two national rooms in the Cathedral ...,5,there are two national rooms in the cathedral ...,149
3,3,do6,Female,Russian,2164,190,459,There are two nation rooms in the Cathedral of...,5,there are two nation rooms in the cathedral of...,187
4,4,bv5,Male,Arabic,151,4,311,"""Not all learning takes place in the classroom...",5,not all learning takes place in the classroom ...,136
...,...,...,...,...,...,...,...,...,...,...,...
17144,46214,bh8,Male,Japanese,48034,6087,107,I received my Medical Doctor license in Japan ...,3,i received my medical doctor license in japan ...,54
17145,46215,bh8,Male,Japanese,48293,6119,138,I introduce my ideal home to you quickly. This...,3,i introduce my ideal home to you quickly this ...,88
17146,46217,cz3,Female,Korean,47451,6027,102,I will put 5 items in time capsule such as som...,3,i will put 5 items in time capsule such as som...,67
17147,46218,cz3,Female,Korean,48001,6087,104,I have studied Visual Design since 2002. I gra...,3,i have studied visual design since 2002 i grad...,57


# Cleaning Loose Dataset

In [42]:
drop_loose=loose.dropna()

sorted_loose=drop_loose.sort_values(by=['anon_id']).reset_index(drop=True)
sorted_loose

,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts)
0,34686.0,aa0,Spanish,5.0,4497.0,109.0,"Barber, chef by profession, but an expert on a..."
1,32129.0,aa0,Spanish,5.0,4199.0,190.0,"The article ""English as Co star"" support the f..."
2,33302.0,aa0,Spanish,5.0,4346.0,193.0,"In this article the authors Goleman, Kaufman a..."
3,33308.0,aa0,Spanish,5.0,4347.0,170.0,"Flow in the sense expressed in the text ""The C..."
4,32572.0,aa0,Spanish,5.0,4260.0,85.0,"Bill Gates, in this conference, explained two ..."
...,...,...,...,...,...,...,...
1805,32406.0,ha2,Korean,5.0,4273.0,217.0,Pittsburgh had a big snowstorm recently. A lot...
1806,32326.0,ha2,Korean,5.0,4256.0,170.0,"Korea, which is connected to China, belongs to..."
1807,32077.0,ha2,Korean,5.0,4197.0,195.0,Koko is a big female gorilla who was born in S...
1808,29631.0,ha2,Korean,4.0,3943.0,283.0,A natural disaster is the effect of the natura...


In [43]:
index=0
textcount_loose=sorted_loose.copy()

textcount_loose['processed_text']=pd.NaT
textcount_loose['wordtype_len']=pd.NaT
words=[]
for text in textcount_loose['text3 (edits made to fix word counts)']:
    uwords=[]
    text=text.replace('\n',' ')
    nopunc=text.translate(str.maketrans('','',string.punctuation))
    nopunc=nopunc.lower()
    textcount_loose.at[index,'processed_text']=nopunc
    words=nopunc.strip().split(" ")
    for word in words:
        if word=='':
            words.remove(word)
        elif word not in uwords:
            uwords.append(word)
    textcount_loose.at[index,'text3_len']=len(words)
    textcount_loose.at[index,'wordtype_len']=len(uwords)
    index+=1

In [44]:
textcount_loose

,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts),processed_text,wordtype_len
0,34686.0,aa0,Spanish,5.0,4497.0,109.0,"Barber, chef by profession, but an expert on a...",barber chef by profession but an expert on agr...,79
1,32129.0,aa0,Spanish,5.0,4199.0,190.0,"The article ""English as Co star"" support the f...",the article english as co star support the fac...,117
2,33302.0,aa0,Spanish,5.0,4346.0,193.0,"In this article the authors Goleman, Kaufman a...",in this article the authors goleman kaufman an...,122
3,33308.0,aa0,Spanish,5.0,4347.0,170.0,"Flow in the sense expressed in the text ""The C...",flow in the sense expressed in the text the cr...,103
4,32572.0,aa0,Spanish,5.0,4260.0,85.0,"Bill Gates, in this conference, explained two ...",bill gates in this conference explained two bi...,66
...,...,...,...,...,...,...,...,...,...
1805,32406.0,ha2,Korean,5.0,4273.0,217.0,Pittsburgh had a big snowstorm recently. A lot...,pittsburgh had a big snowstorm recently a lot ...,129
1806,32326.0,ha2,Korean,5.0,4256.0,170.0,"Korea, which is connected to China, belongs to...",korea which is connected to china belongs to t...,101
1807,32077.0,ha2,Korean,5.0,4197.0,195.0,Koko is a big female gorilla who was born in S...,koko is a big female gorilla who was born in s...,115
1808,29631.0,ha2,Korean,4.0,3943.0,283.0,A natural disaster is the effect of the natura...,a natural disaster is the effect of the natura...,135


# Level Grouping Whole Dataset

In [45]:
sac_lvls=stu_ans_crs[['anon_id','native_language','level_id']]
sac_lvls_rkc=sac_lvls

index=0
for i in sac_lvls_rkc.native_language:
    if (i != 'Korean' and i != 'Spanish' and i != 'Chinese'):
        sac_lvls_rkc=sac_lvls_rkc.drop(index)
    index+=1
    
# allows us to see what levels each anon_id participated in 
lvl_list=sac_lvls_rkc.groupby('anon_id').agg(n_uniq=('level_id','nunique'), lvl_nums=('level_id',get_uniques))
lvl_list=lvl_list.reset_index()

In [46]:
# will create a list of only the ids that have levels 3, 4, and 5 in their lvl_nums column
all_three=[]
ind=0
for i in lvl_list.anon_id:
    if ((3 in lvl_list.iat[ind,2]) and (4 in lvl_list.iat[ind,2]) and (5 in lvl_list.iat[ind,2])):
        append=lvl_list.at[ind, 'anon_id']
        all_three.append(append)
    ind+=1

all_three.sort()
len(all_three)

42

In [47]:
# will create a list of only the ids that have levels 3 and 4 in their lvl_nums column
lvls34=[]
ind=0
for i in lvl_list.anon_id:
    if ((3 in lvl_list.iat[ind,2]) and (4 in lvl_list.iat[ind,2])):
        append=lvl_list.at[ind, 'anon_id']
        lvls34.append(append)
    ind+=1

lvls34.sort()
len(lvls34)

111

In [48]:
# will create a list of only the ids that have levels 3 and 4 in their lvl_nums column
lvls45=[]
ind=0
for i in lvl_list.anon_id:
    if ((4 in lvl_list.iat[ind,2]) and (5 in lvl_list.iat[ind,2])):
        append=lvl_list.at[ind, 'anon_id']
        lvls45.append(append)
    ind+=1

lvls45.sort()
len(lvls45)

154

# Level Grouping Loose Dataset

In [49]:
# this code gets how many times an id appeared (n_anon_ids) as well as an array of what levels they participated in (lvl_nums)
clean_ids_lvl_loose=drop_loose.groupby('anon_id').agg(
    n_anon_ids_loose=('level_id', 'nunique'),
    lvl_nums_loose=('level_id', get_uniques)
)

#changes the clean_ids_lvl's index so we can access the anon_id easier
ind_loose= clean_ids_lvl_loose.reset_index()
ind_loose

,anon_id,n_anon_ids_loose,lvl_nums_loose
0,aa0,1,[5.0]
1,aa3,1,[4.0]
2,aa8,2,"[4.0, 5.0]"
3,aa9,2,"[3.0, 4.0]"
4,ab6,1,[4.0]
...,...,...,...
278,gz2,2,"[4.0, 5.0]"
279,gz5,1,[4.0]
280,gz7,1,[4.0]
281,ha0,1,[5.0]


In [50]:
# for loop that checks if lvl_nums has all three levels in it (3,4, and 5)
# if there are three values in lvl_nums, the anon_id is appended to the all_three list
ind=0
all_three_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==3:
        append=ind_loose.at[ind,'anon_id']
        all_three_loose.append(append)
    ind=ind+1

all_three_loose.sort()
len(all_three_loose)

8

In [51]:
# creates list of participants in loose sheet that are in level 3 and 4
ind=0
in34_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==2:
        if (3 in ind_loose.iat[ind,2] and 4 in ind_loose.iat[ind,2]):
            append=ind_loose.at[ind,'anon_id']
            in34_loose.append(append)
    elif ind_loose.iat[ind,1]==3:
        append=ind_loose.at[ind,'anon_id']
        in34_loose.append(append)
    ind=ind+1
in34_loose.sort()
print(len(in34_loose))

28


In [52]:
# creates list of participants in loose sheet that are in level 4 and 5
ind=0
in45_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==2:
        if (4 in ind_loose.iat[ind,2] and 5 in ind_loose.iat[ind,2]):
            append=ind_loose.at[ind,'anon_id']
            in45_loose.append(append)
    elif ind_loose.iat[ind,1]==3:
        append=ind_loose.at[ind,'anon_id']
        in45_loose.append(append)
    ind=ind+1
in45_loose.sort()
print(len(in45_loose))

61


# Vocabulary Sophistication

In [53]:
#ranked word list containing most commonly used words in the English language
word_list=pd.read_csv("unigram_freq.csv")

In [54]:
word_list

,word,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698
...,...,...
333328,gooek,12711
333329,gooddg,12711
333330,gooblle,12711
333331,gollgo,12711


In [55]:
#creates a list of words from the word_list df
def thousandcount(df, word_list, count,limit):
    while(count<limit):
        word=str(df.iat[count,0])
        word_list.append(word)
        count+=1
    return count

In [56]:
#will create a list of words out of the answer text then check those words with whatever thousand list is passed
#if a word from the answer is also in the thousand list, then it's added to a list. the length of this list is
#the number of words from the thousands list that occurs in the answer text and that's appended to a new column for the thousand list
def vocSophCols(df, col_name, col2_name, text_col, thouslist):
    index=0
    df[col_name]=pd.NaT
    df[col2_name]=pd.NaT
    df["Words "+col2_name]=pd.NaT
    for text in df[text_col]:
        twords=[]
        uwords=[]
        words=[]
        words=text.strip().split(" ")
        for word in words:
            if word in thouslist:
                twords.append(word)
                if word not in uwords:
                    uwords.append(word)
        df.at[index,col_name]=len(twords)
        df.at[index,col2_name]=len(uwords)
        df.at[index,"Words "+col2_name]=str(uwords)
        index+=1

In [57]:
#creating list of the first,second,third, etc thousand words in the most common words list
firstk,secondk,thirdk,fourthk,fifthk,sixthk,seventhk,eighthk,ninthk,tenthk,rest=[],[],[],[],[],[],[],[],[],[],[]
count=0

count=thousandcount(word_list,firstk,count,1000)
count=thousandcount(word_list,secondk,count,2000)
count=thousandcount(word_list, thirdk, count,3000)
count=thousandcount(word_list, fourthk, count,4000)
count=thousandcount(word_list, fifthk, count,5000)
count=thousandcount(word_list, sixthk, count,6000)
count=thousandcount(word_list, seventhk, count,7000)
count=thousandcount(word_list,eighthk,count, 8000)
count=thousandcount(word_list,ninthk,count,9000)
count=thousandcount(word_list,tenthk,count, 10000)
#count=thousandcount(word_list,rest,count,333333)

In [58]:
#testing
text="the of a stuff online stuff glapagos"
twords=[]
words=text.strip().split(" ")
for word in words:
    if word in firstk:
        twords.append(word)
print(twords)
len(twords)

['the', 'of', 'a', 'stuff', 'online', 'stuff']


6

# Whole Dataset

In [59]:
#creating a list of all the ids in all the groups
all_groups=[]
for i in all_three: # and lvls34 and lvls45:
    if i not in all_groups:
        all_groups.append(i)
for i in lvls34:
    if i not in all_groups:
        all_groups.append(i)
for i in lvls45:
    if i not in all_groups:
        all_groups.append(i)

In [60]:
stu_ans_crs

,index,anon_id,gender,native_language,answer_id,question_id,text_len,text,level_id,processed_text,wordtype_len
0,0,do6,Female,Russian,150,4,299,Some people prefer eat out and some like doing...,5,some people prefer eat out and some like doing...,125
1,1,do6,Female,Russian,1221,97,288,My opinion is that a person does need educatio...,5,my opinion is that a person does need educatio...,112
2,2,do6,Female,Russian,1957,189,317,There are two national rooms in the Cathedral ...,5,there are two national rooms in the cathedral ...,149
3,3,do6,Female,Russian,2164,190,459,There are two nation rooms in the Cathedral of...,5,there are two nation rooms in the cathedral of...,187
4,4,bv5,Male,Arabic,151,4,311,"""Not all learning takes place in the classroom...",5,not all learning takes place in the classroom ...,136
...,...,...,...,...,...,...,...,...,...,...,...
17144,46214,bh8,Male,Japanese,48034,6087,107,I received my Medical Doctor license in Japan ...,3,i received my medical doctor license in japan ...,54
17145,46215,bh8,Male,Japanese,48293,6119,138,I introduce my ideal home to you quickly. This...,3,i introduce my ideal home to you quickly this ...,88
17146,46217,cz3,Female,Korean,47451,6027,102,I will put 5 items in time capsule such as som...,3,i will put 5 items in time capsule such as som...,67
17147,46218,cz3,Female,Korean,48001,6087,104,I have studied Visual Design since 2002. I gra...,3,i have studied visual design since 2002 i grad...,57


In [61]:
#removing the ids that aren't in any of the level groups
sort_sac=stu_ans_crs.sort_values(by=['anon_id']).reset_index(drop=True)
sort_sac=sort_sac.drop(columns=['index','answer_id','gender'])
just_groups=sort_sac.copy()
drops=[]
ind=0
for i in sort_sac['anon_id']:
    if i not in all_groups:
        drops.append(ind)
    ind+=1

just_groups=just_groups.drop(labels=drops,axis=0).reset_index()
just_groups

,index,anon_id,native_language,question_id,text_len,text,level_id,processed_text,wordtype_len
0,60,aa8,Korean,1258,770,College students are usually very busy during ...,5,college students are usually very busy during ...,232
1,61,aa8,Korean,1511,64,I think it is strange to throw away something ...,4,i think it is strange to throw away something ...,51
2,62,aa8,Korean,1184,809,College students are usually very busy during ...,5,college students are usually very busy during ...,237
3,63,aa8,Korean,1219,343,I thought this was homework. \nT.T I could not...,5,i thought this was homework tt i could not th...,150
4,64,aa8,Korean,1138,137,I. I think using a schedule book is a good way...,5,i i think using a schedule book is a good way ...,63
...,...,...,...,...,...,...,...,...,...
5221,17074,hb4,Korean,160,397,Cats and dogs are the most popular pets in the...,4,cats and dogs are the most popular pets in the...,172
5222,17075,hb4,Korean,203,440,Cats and dogs are the most popular pets in the...,4,cats and dogs are the most popular pets in the...,183
5223,17076,hb4,Korean,117,405,"Global Warming\n\nIn the greenhouse, plants ca...",4,global warming in the greenhouse plants can l...,199
5224,17077,hb4,Korean,472,110,"My hometown is Seoul, which is capital city of...",4,my hometown is seoul which is capital city of ...,70


In [62]:
vocSophCols(just_groups, '1k','WT 1k','processed_text',firstk)
vocSophCols(just_groups,'2k','WT 2k','processed_text',secondk)
vocSophCols(just_groups,'3k','WT 3k','processed_text',thirdk)
vocSophCols(just_groups,'4k','WT 4k','processed_text',fourthk)
vocSophCols(just_groups,'5k','WT 5k','processed_text',fifthk)
vocSophCols(just_groups,'6k','WT 6k','processed_text',sixthk)
vocSophCols(just_groups,'7k','WT 7k','processed_text',seventhk)
vocSophCols(just_groups,'8k','WT 8k','processed_text',eighthk)
vocSophCols(just_groups,'9k','WT 9k','processed_text',ninthk)
vocSophCols(just_groups,'10k','WT 10k','processed_text',tenthk)

In [64]:
index=0
just_groups['10k+']=pd.NaT
for length in just_groups.text_len:
    k1=just_groups.at[index,'1k']
    k2=just_groups.at[index,'2k']
    k3=just_groups.at[index,'3k']
    k4=just_groups.at[index,'4k']
    k5=just_groups.at[index,'5k']
    k6=just_groups.at[index,'6k']
    k7=just_groups.at[index,'7k']
    k8=just_groups.at[index,'8k']
    k9=just_groups.at[index,'9k']
    k10=just_groups.at[index,'10k']
    total=k1+k2+k3+k4+k5+k6+k7+k8+k9+k10
    remaining=length-total
    just_groups.at[index, '10k+']=remaining
    
    lengthwt=just_groups.at[index,'wordtype_len']
    k1wt=just_groups.at[index,'WT 1k']
    k2wt=just_groups.at[index,'WT 2k']
    k3wt=just_groups.at[index,'WT 3k']
    k4wt=just_groups.at[index,'WT 4k']
    k5wt=just_groups.at[index,'WT 5k']
    k6wt=just_groups.at[index,'WT 6k']
    k7wt=just_groups.at[index,'WT 7k']
    k8wt=just_groups.at[index,'WT 8k']
    k9wt=just_groups.at[index,'WT 9k']
    k10wt=just_groups.at[index,'WT 10k']
    totalwt=k1wt+k2wt+k3wt+k4wt+k5wt+k6wt+k7wt+k8wt+k9wt+k10wt
    remainingwt=lengthwt-totalwt
    just_groups.at[index,'WT 10k+']=remainingwt
    index+=1

In [65]:
just_groups

,index,anon_id,native_language,question_id,text_len,text,level_id,processed_text,wordtype_len,1k,...,WT 8k,Words WT 8k,9k,WT 9k,Words WT 9k,10k,WT 10k,Words WT 10k,10k+,WT 10k+
0,60,aa8,Korean,1258,770,College students are usually very busy during ...,5,college students are usually very busy during ...,232,606,...,2,"['cant', 'scanning']",7,3,"['quizzes', 'efficiently', 'divide']",4,3,"['concentrate', 'bother', 'qt']",28,14.0
1,61,aa8,Korean,1511,64,I think it is strange to throw away something ...,4,i think it is strange to throw away something ...,51,43,...,0,[],1,1,['predict'],0,0,[],5,5.0
2,62,aa8,Korean,1184,809,College students are usually very busy during ...,5,college students are usually very busy during ...,237,665,...,3,"['cant', 'nervous', 'organize']",1,1,['efficiently'],2,2,"['concentrate', 'bother']",18,10.0
3,63,aa8,Korean,1219,343,I thought this was homework. \nT.T I could not...,5,i thought this was homework tt i could not th...,150,271,...,2,"['grammar', 'expressions']",0,0,[],1,1,['concentrate'],19,8.0
4,64,aa8,Korean,1138,137,I. I think using a schedule book is a good way...,5,i i think using a schedule book is a good way ...,63,108,...,1,['organize'],0,0,[],0,0,[],11,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5221,17074,hb4,Korean,160,397,Cats and dogs are the most popular pets in the...,4,cats and dogs are the most popular pets in the...,172,280,...,2,"['cant', 'lonely']",1,1,['differently'],0,0,[],12,12.0
5222,17075,hb4,Korean,203,440,Cats and dogs are the most popular pets in the...,4,cats and dogs are the most popular pets in the...,183,313,...,3,"['lonely', 'aggressive', 'cant']",2,1,['differently'],0,0,[],14,13.0
5223,17076,hb4,Korean,117,405,"Global Warming\n\nIn the greenhouse, plants ca...",4,global warming in the greenhouse plants can l...,199,241,...,1,['cant'],12,4,"['warming', 'phenomenon', 'rays', 'centuries']",12,3,"['greenhouse', 'refrigerator', 'ozone']",37,29.0
5224,17077,hb4,Korean,472,110,"My hometown is Seoul, which is capital city of...",4,my hometown is seoul which is capital city of ...,70,82,...,1,['neighbors'],1,1,['lobby'],1,1,['hometown'],6,3.0


# Loose Dataset

In [68]:
loose_soph=textcount_loose.sort_values(by=['anon_id'])
loose_soph

,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts),processed_text,wordtype_len
0,34686.0,aa0,Spanish,5.0,4497.0,109.0,"Barber, chef by profession, but an expert on a...",barber chef by profession but an expert on agr...,79
1,32129.0,aa0,Spanish,5.0,4199.0,190.0,"The article ""English as Co star"" support the f...",the article english as co star support the fac...,117
2,33302.0,aa0,Spanish,5.0,4346.0,193.0,"In this article the authors Goleman, Kaufman a...",in this article the authors goleman kaufman an...,122
3,33308.0,aa0,Spanish,5.0,4347.0,170.0,"Flow in the sense expressed in the text ""The C...",flow in the sense expressed in the text the cr...,103
4,32572.0,aa0,Spanish,5.0,4260.0,85.0,"Bill Gates, in this conference, explained two ...",bill gates in this conference explained two bi...,66
...,...,...,...,...,...,...,...,...,...
1792,32816.0,ha2,Korean,5.0,4305.0,177.0,"In Korea, a lot of parents who want their chil...",in korea a lot of parents who want their child...,100
1791,29184.0,ha2,Korean,4.0,3852.0,69.0,The importance of women is getting greater in ...,the importance of women is getting greater in ...,53
1808,29631.0,ha2,Korean,4.0,3943.0,283.0,A natural disaster is the effect of the natura...,a natural disaster is the effect of the natura...,135
1799,35728.0,ha2,Korean,5.0,4604.0,315.0,"Fresh air, the npr's program, invited Ben Stil...",fresh air the nprs program invited ben stiller...,159


In [ ]:
vocSophCols(loose_soph, '1k','WT 1k','processed_text',firstk)
vocSophCols(loose_soph,'2k','WT 2k','processed_text',secondk)
vocSophCols(loose_soph,'3k','WT 3k','processed_text',thirdk)
vocSophCols(loose_soph,'4k','WT 4k','processed_text',fourthk)
vocSophCols(loose_soph,'5k','WT 5k','processed_text',fifthk)
vocSophCols(loose_soph,'6k','WT 6k','processed_text',sixthk)
vocSophCols(loose_soph,'7k','WT 7k','processed_text',seventhk)
vocSophCols(loose_soph,'8k','WT 8k','processed_text',eighthk)
vocSophCols(loose_soph,'9k','WT 9k','processed_text',ninthk)
vocSophCols(loose_soph,'10k','WT 10k','processed_text',tenthk)

In [ ]:
index=0
loose_soph['10k+']=pd.NaT
for length in loose_soph.text3_len:
    k1=loose_soph.at[index,'1k']
    k2=loose_soph.at[index,'2k']
    k3=loose_soph.at[index,'3k']
    k4=loose_soph.at[index,'4k']
    k5=loose_soph.at[index,'5k']
    k6=loose_soph.at[index,'6k']
    k7=loose_soph.at[index,'7k']
    k8=loose_soph.at[index,'8k']
    k9=loose_soph.at[index,'9k']
    k10=loose_soph.at[index,'10k']
    total=k1+k2+k3+k4+k5+k6+k7+k8+k9+k10
    remaining=length-total
    loose_soph.at[index, '10k+']=remaining
    
    lengthwt=loose_soph.at[index,'wordtype_len']
    k1wt=loose_soph.at[index,'WT 1k']
    k2wt=loose_soph.at[index,'WT 2k']
    k3wt=loose_soph.at[index,'WT 3k']
    k4wt=loose_soph.at[index,'WT 4k']
    k5wt=loose_soph.at[index,'WT 5k']
    k6wt=loose_soph.at[index,'WT 6k']
    k7wt=loose_soph.at[index,'WT 7k']
    k8wt=loose_soph.at[index,'WT 8k']
    k9wt=loose_soph.at[index,'WT 9k']
    k10wt=loose_soph.at[index,'WT 10k']
    totalwt=k1wt+k2wt+k3wt+k4wt+k5wt+k6wt+k7wt+k8wt+k9wt+k10wt
    remainingwt=lengthwt-totalwt
    loose_soph.at[index,'WT 10k+']=remainingwt
    index+=1

In [ ]:
loose_soph

In [ ]:
# finds the average of all the columns 
def vocDist(df, lang_col, lvl, lang, thoulist):
    index=0
    count=0
    thoucount=0
    for i in df.anon_id:
        if df.at[index,'level_id']==lvl and df.at[index,lang_col]==lang:
            thoucount+=df.at[index,thoulist]
            count+=1
        index+=1
    return thoucount/count